In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [8]:
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.7)
output = llm("I want to open an Italian food restuarant, suggest fancy names for this.")
print(output)




1. "Cibo Italiano"
2. "Bella Italia"
3. "La Dolce Vita"
4. "Trattoria Elegante"
5. "Ristorante Di Lusso"
6. "Cucina Classica"
7. "Al Dente"
8. "Fresco Italiano"
9. "Gusto Italiano"
10. "Vivace Ristorante"
11. "Sapori d'Italia"
12. "Ristorante di Roma"
13. "Grazie Italiano"
14. "Sapore di Toscana"
15. "Bistro Fiore"
16. "Bella Tavola"
17. "La Cucina Bella"
18. "Ristorante Amore"
19. "Bella Cucina"
20. "Cucina di Lusso"


### **1.Chain** (Runs a single prompt with a language model. You give it input, it gives you output.)


In [9]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open an {cuisine} food restuarant, suggest a fancy name for this."
)

prompt_template_name.format(cuisine="Mexican")

'I want to open an Mexican food restuarant, suggest a fancy name for this.'

In [11]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Gujarati")

'\n\n"Kathiyawadi Delights"'

### **2.Simple Sequential Chain** (Connects multiple chains in a row. The output of one chain becomes the input for the next. It only passes a single string between chains.)

In [18]:
llm = OpenAI(temperature = 0.6)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open an {cuisine} food restuarant, suggest a fancy name for this."
)
name_chain = LLMChain(llm = llm, prompt = prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables=['restuarant_name'],
    template="Suggest some menu items for {restuarant_name}. Return them as a comma seperated list."
) 
food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items)

In [ ]:
from langchain.chains import SimpleSequentialChain

seq_chain = SimpleSequentialChain(chains = [name_chain, food_items_chain]) # how this works is basically, output of name_chain 
res = seq_chain.run("Indian")                                              # is taken as the input for food_items_chain
print(res)



1. Tandoori Chicken
2. Vegetable Samosas
3. Butter Chicken
4. Lamb Rogan Josh
5. Palak Paneer
6. Chicken Tikka Masala
7. Chana Masala
8. Garlic Naan
9. Mango Lassi
10. Biryani Rice
11. Aloo Gobi
12. Malai Kofta
13. Dal Makhani
14. Mango Chutney
15. Gulab Jamun

Tandoori Chicken, Vegetable Samosas, Butter Chicken, Lamb Rogan Josh, Palak Paneer, Chicken Tikka Masala, Chana Masala, Garlic Naan, Mango Lassi, Biryani Rice, Aloo Gobi, Malai Kofta, Dal Makhani, Mango Chutney, Gulab Jamun


### **3.Sequential Chain** (Connects multiple chains, but can handle multiple inputs and outputs (using dictionaries). It’s more flexible and can pass several variables between chains.)

In [24]:
llm = OpenAI(temperature = 0.6)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open an {cuisine} food restuarant, suggest a fancy name for this."
)
name_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key = "restuarant_name")

prompt_template_items = PromptTemplate(
    input_variables=['restuarant_name'],
    template="Suggest some menu items for {restuarant_name}. Return them as a comma seperated list."
) 
food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items, output_key = "menu_items")

In [26]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restuarant_name', 'menu_items']
)
chain({'cuisine': 'Indian'}) # need to provide the input varibales as a dict as we might have mutliple inputs in the case of SequentialChain

/var/folders/fj/kvkr7bks3yx9yvxbxh2hc9hc0000gn/T/ipykernel_72196/1238844908.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({'cuisine': 'Indian'}) # need to provide the input varibales as a dict as we might have mutliple inputs in the case of SequentialChain


{'cuisine': 'Indian',
 'restuarant_name': '\n\n"Saffron Spice Palace"',
 'menu_items': '\n\n1. Saffron Chicken Tikka Masala\n2. Lamb Korma\n3. Vegetable Biryani\n4. Tandoori Shrimp Skewers\n5. Paneer Butter Masala\n6. Dal Makhani\n7. Chicken Vindaloo\n8. Aloo Gobi\n9. Tandoori Naan\n10. Mango Lassi\n11. Samosas\n12. Palak Paneer\n13. Lamb Rogan Josh\n14. Chana Masala\n15. Gulab Jamun dessert.'}

## **Agents**

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose = True
)

agent.run("When was Salman Khan born? What is his age right now in 2025?")



> Entering new AgentExecutor chain...
 We should use Wikipedia to find out when Salman Khan was born
Action: wikipedia
Action Input: Salman Khan
Observation: Page: Salman Khan
Summary: Salman Salim Khan (Hindi: [səlˈmɑːn ˈxɑːn]; born 27 December 1965) is an Indian actor, film producer, and television personality who predominantly works in Hindi films. In a career spanning over three decades, his awards include two National Film Awards as a film producer, and two Filmfare Awards as an actor. He has been cited in the media as one of the most popular and commercially successful actors of Indian cinema. Forbes included him in listings of the highest-paid celebrities in the world, in 2015 and 2018. He has starred in the annual highest-grossing Hindi films of 10 individual years, the highest for any actor.
Khan began his acting career with a supporting role in Biwi Ho To Aisi (1988), followed by his breakthrough with a leading role in Sooraj Barjatya's romantic drama Maine Pyar Kiya (1989)

'Salman Khan was born on December 27, 1965, making him 60 years old in 2025.'